In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine
import datetime as datetime
import warnings
warnings.filterwarnings('ignore')
import io


In [3]:
# Read csv data
data_file = "Resources/consolidated_coin_data.csv"
crypto_df = pd.read_csv(data_file)
crypto_df.head()

,Currency,Date,Open,High,Low,Close,Volume,Market Cap
0,ripple,24-Apr-19,0.32,0.32,0.30,0.30,1517791002,1.269888e+10
1,ripple,23-Apr-19,0.32,0.33,0.32,0.32,1077333990,1.349293e+10
2,ripple,22-Apr-19,0.32,0.33,0.32,0.32,1131094080,1.360682e+10
3,ripple,21-Apr-19,0.33,0.33,0.32,0.32,1005803846,1.353341e+10
4,ripple,20-Apr-19,0.33,0.33,0.32,0.33,931570799,1.378638e+10


In [4]:
#Read json Data
data2_file = "Resources/convertcsv.json"
cryptoj_df = pd.read_json(data2_file)
cryptoj_df.head()

,close,close_ratio,date,high,low,market,name,open,ranknow,slug,spread,symbol,volume
0,134.21,0.5438,2013-04-28,135.98,132.10,1488566728,Bitcoin,135.30,1,bitcoin,3.88,BTC,0
1,144.54,0.7813,2013-04-29,147.49,134.00,1603768865,Bitcoin,134.44,1,bitcoin,13.49,BTC,0
2,139.00,0.3843,2013-04-30,146.93,134.05,1542813125,Bitcoin,144.00,1,bitcoin,12.88,BTC,0
3,116.99,0.2882,2013-05-01,139.89,107.72,1298954594,Bitcoin,139.00,1,bitcoin,32.17,BTC,0
4,105.21,0.3881,2013-05-02,125.60,92.28,1168517495,Bitcoin,116.38,1,bitcoin,33.32,BTC,0


In [5]:
# take this json file and delete columns already listed:
# "Currency", "Date_open", "Open_price", "High", "Low", "Close_price", "Volume", "Market_Cap" 
json_file_df=cryptoj_df.drop(["name", "date", "open", "high", "low", "close", "volume", "market"], axis=1)
json_file_df.head()

,close_ratio,ranknow,slug,spread,symbol
0,0.5438,1,bitcoin,3.88,BTC
1,0.7813,1,bitcoin,13.49,BTC
2,0.3843,1,bitcoin,12.88,BTC
3,0.2882,1,bitcoin,32.17,BTC
4,0.3881,1,bitcoin,33.32,BTC


In [6]:
# Rename the slug column header for json file
json_file_df.columns =['Close_Ratio','Ranknow','Currency','Spread','System']
json_file_df.head()

,Close_Ratio,Ranknow,Currency,Spread,System
0,0.5438,1,bitcoin,3.88,BTC
1,0.7813,1,bitcoin,13.49,BTC
2,0.3843,1,bitcoin,12.88,BTC
3,0.2882,1,bitcoin,32.17,BTC
4,0.3881,1,bitcoin,33.32,BTC


In [7]:
#print csv data type
crypto_df["Date"] = crypto_df["Date"].astype("datetime64")
crypto_df = crypto_df.convert_objects(convert_numeric=True)

crypto_df.dtypes

Currency                object
Date            datetime64[ns]
 Open                  float64
 High                  float64
 Low                   float64
 Close                 float64
 Volume                float64
 Market Cap            float64
dtype: object

In [8]:

crypto_df.columns = ["Currency", "Date_open", "Open_price", "High", "Low", "Close_price", "Volume", "Market_Cap"]
crypto_df= crypto_df.dropna()
crypto_df.head()

,Currency,Date_open,Open_price,High,Low,Close_price,Volume,Market_Cap
0,ripple,2019-04-24,0.32,0.32,0.30,0.30,1.517791e+09,1.269888e+10
1,ripple,2019-04-23,0.32,0.33,0.32,0.32,1.077334e+09,1.349293e+10
2,ripple,2019-04-22,0.32,0.33,0.32,0.32,1.131094e+09,1.360682e+10
3,ripple,2019-04-21,0.33,0.33,0.32,0.32,1.005804e+09,1.353341e+10
4,ripple,2019-04-20,0.33,0.33,0.32,0.33,9.315708e+08,1.378638e+10


In [22]:
rds_connection_string = "mysql+mysqlconnector://root:<password>@localhost:3306/cryptocurrency_db"

engine = create_engine(f'{rds_connection_string}')

In [23]:
engine.table_names()

['data', 'ranks']

In [24]:
crypto_df.to_sql(name='data', con=engine, if_exists='append', index=False)

In [25]:
json_file_df.to_sql(name='ranks', con=engine, if_exists='append', index=False)

In [27]:
pd.read_sql_query('select * from data', con=engine).head()

,Currency,Date_open,Open_price,High,Low,Close_price,Volume,Market_Cap
0,ripple,2019-04-24,0.32,0.32,0.30,0.30,1.517790e+09,1.269890e+10
1,ripple,2019-04-23,0.32,0.33,0.32,0.32,1.077330e+09,1.349290e+10
2,ripple,2019-04-22,0.32,0.33,0.32,0.32,1.131090e+09,1.360680e+10
3,ripple,2019-04-21,0.33,0.33,0.32,0.32,1.005800e+09,1.353340e+10
4,ripple,2019-04-20,0.33,0.33,0.32,0.33,9.315710e+08,1.378640e+10


In [28]:
pd.read_sql_query('select * from ranks', con=engine).head()

,Close_Ratio,Ranknow,Currency,Spread,System
0,1,1,bitcoin,4,BTC
1,1,1,bitcoin,13,BTC
2,0,1,bitcoin,13,BTC
3,0,1,bitcoin,32,BTC
4,0,1,bitcoin,33,BTC


In [29]:
 pd.merge(crypto_df, json_file_df, on='Currency', how='inner')

,Currency,Date_open,Open_price,High,Low,Close_price,Volume,Market_Cap,Close_Ratio,Ranknow,Spread,System
0,ripple,2019-04-24,0.32,0.32,0.3,0.3,1.517791e+09,1.269888e+10,0.1509,2,0.0,XRP
1,ripple,2019-04-24,0.32,0.32,0.3,0.3,1.517791e+09,1.269888e+10,0.0000,2,0.0,XRP
2,ripple,2019-04-24,0.32,0.32,0.3,0.3,1.517791e+09,1.269888e+10,0.0494,2,0.0,XRP
3,ripple,2019-04-24,0.32,0.32,0.3,0.3,1.517791e+09,1.269888e+10,0.2407,2,0.0,XRP
4,ripple,2019-04-24,0.32,0.32,0.3,0.3,1.517791e+09,1.269888e+10,0.3173,2,0.0,XRP


In [ ]:
 pd.merge(crypto_df, json_file_df, on='Currency', how='inner')\
    .to_sql(name='complete_data', con=engine, if_exists='append',\
            index=False)